Now that we have fixed and generated three feature subsets
1. non-lagged + lagged textual features
2. lagged {target,item,shop} + non-lagged basic categories
3. lagged features within shop

and three first level classifiers types for each
* a.  CatBoost
* b. RidgeCV 
* c. Random Forest (sklearn) 

we search for hyperarameters that are used for predicting a month 
based on twelve month history, with one month gap between training and prediction periods.

This is a compromise of the prediction quality on the other hand, and not having the prediction 
quality and optimal hyperparameters vary too much over the training period when generating the first level predictions as input features of second stacking level.

The search for hyperparameters is problematic in whole because the chosen validation scheme is lacking. There may not be
too much that can be done, because the validation data necessarily has different distribution as the actual testing data.
This is because the temporal nature of the prediction problem. The distributions slowly drift during cause of time. Therefore, 
it is good to have the validation period temporally close to the test period. On the other hand, data analysis shows strong seasonal=(yearly) effects. 
Predicting October sales based on previous year simply is a very different problem to predicting December sales, as sales figures seem to peak strongly in December and have special characteristics.

We decide to search for such hyperparameters that maximise the quality of predictions (with
reasonable computational burden) in the hold-out validation data of Oct 2015. This is despite the fact that we have seen in examples that
such optimal model hyperparameters do not result in optimal prediction quality for Dec 2015.
We specifically do not search for such hyperparameters (via a coross-validation scheme) that would maximise the quality of predictions during
the training period, as the value of temporally distant predictions is questionable after because of the distribution shift throughtime.


The parameters are used for
a) creating submissions for ensembling using simple schemes
b) generating level 2 input features for a stacking algorithm



In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm as lgb
from sklearn.metrics import r2_score
import catboost
import gc

from catboost import CatBoostRegressor, Pool

import re
import os

for p in [np, pd, scipy, sklearn, lgb, catboost]:
    print (p.__name__, p.__version__)
    
DATA_FOLDER = 'competitive-data-science-predict-future-sales'
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

index_cols=['item_id','shop_id','date_block_num']
date_block_val = 33
date_block_test = 35 # Dec 2015

test2submission_mapping_generated = False

numpy 1.18.1
pandas 0.25.3
scipy 1.4.1
sklearn 0.22.1
lightgbm 2.3.1
catboost 0.22


In [2]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [3]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')

In [4]:
def clipped_rmse(gt, predicted,clip_min=0, clip_max=20):
  target=np.minimum(np.maximum(gt,clip_min), clip_max)
  return np.sqrt((target-predicted)**2).mean()

# Feature set 1: non-lagged and lagged basic categories

In [5]:
# load data

all_data = pd.read_csv(os.path.join(DATA_FOLDER, 'feature_set_basic.csv'))

dates=all_data['date_block_num']

dates_train = (dates>= date_block_val - 13) & (dates<= date_block_val - 2)
dates_trainval = (dates>= date_block_test - 13) & (dates<= date_block_test - 2)
# y_train = all_data.loc[(dates>= date_block_val - 13) & (dates<= date_block_val - 2), 'target']
# y_trainval = all_data.loc[(dates>= date_block_test - 13) & (dates<= date_block_test - 2), 'target']

y_train=all_data.loc[dates_train, 'target']
y_trainval=all_data.loc[dates_trainval, 'target']
y_val = all_data.loc[dates == date_block_val, 'target']
y_test = all_data.loc[dates == date_block_test, 'target']

to_drop_cols = ['target','date_block_num']

X_train = all_data.loc[dates_train].drop(to_drop_cols, axis=1)
X_trainval = all_data.loc[dates_trainval].drop(to_drop_cols, axis=1)
X_val = all_data.loc[dates == date_block_val].drop(to_drop_cols, axis=1)
X_test = all_data.loc[dates == date_block_test].drop(to_drop_cols, axis=1)

shop_item2submissionid={}
for idx, row in test_spec.iterrows():
    shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])] = row['ID']
    
test_data=all_data.loc[dates == date_block_test, ['shop_id','item_id']]    
    
testidx2submissionidx=np.zeros(test_data.shape[0], dtype=np.int32)
for idx in range(test_data.shape[0]):
    row =test_data.iloc[idx]
    testidx2submissionidx[idx] = shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])]
    
 
#invert the mapping
submissionidx2testidx=np.zeros(test_data.shape[0], dtype=np.int32)
for i in range(test_data.shape[0]):
    submissionidx2testidx[testidx2submissionidx[i]]=i
    
del test_data
gc.collect()    



5

In [6]:
print(y_train.shape)
print(X_train.shape)

(2768948,)
(2768948, 21)


First tackle ridge regression

In [ ]:
from sklearn import linear_model

#model=linear_model.RidgeCV(alphas=np.logspace(-3,13))
model=linear_model.Ridge(alpha=3e6, fit_intercept=False)
model.fit(X_train.to_numpy(), y_train)
pred_val = np.clip(model.predict(X_val.to_numpy()), 0, 20)
#print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))

model.fit(X_trainval.to_numpy(), y_trainval)
pred_test = np.clip(model.predict(X_test.to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-ridge-feature_set_basic.csv')
# LB 1.091266 and 1.088610


In [ ]:
from sklearn import linear_model

#model=linear_model.RidgeCV(alphas=np.logspace(-3,13), normalize=True)
model=linear_model.Ridge(alpha=0.2, normalize=True)
model.fit(X_train.to_numpy(), y_train)
pred_val = np.clip(model.predict(X_val.to_numpy()), 0, 20)
#print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))

model.fit(X_trainval.to_numpy(), y_trainval)
pred_test = np.clip(model.predict(X_test.to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-ridge-normalized-feature_set_basic.csv')
# LB 1.082613 and 1.079229.


In [ ]:
model.alpha_

In [ ]:
# then Catboost
from sklearn import model_selection

reg=CatBoostRegressor(task_type='GPU', loss_function='RMSE', iterations=1200, eta=0.03)

grid = {'depth': range(5,17)}
        
        

reg.grid_search(grid, cv=5, X=X_train.to_numpy(), y=y_train, plot=True)

In [9]:
reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=16, eta=0.3,metric_period=20)
eval_dataset= Pool(X_val,y_val)
reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
#print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))

0:	learn: 3.3958183	test: 5.3132972	best: 5.3132972 (0)	total: 377ms	remaining: 44.8s
20:	learn: 2.3590286	test: 4.9282864	best: 4.9282864 (20)	total: 6.93s	remaining: 32.7s
40:	learn: 2.1750684	test: 4.8845876	best: 4.8845876 (40)	total: 13.5s	remaining: 26.1s
60:	learn: 2.0622922	test: 4.8864402	best: 4.8845876 (40)	total: 20.2s	remaining: 19.5s
80:	learn: 1.9640144	test: 4.8881669	best: 4.8845876 (40)	total: 26.9s	remaining: 13s
100:	learn: 1.8846092	test: 4.8874258	best: 4.8845876 (40)	total: 33.6s	remaining: 6.32s
119:	learn: 1.8466087	test: 4.8860043	best: 4.8845876 (40)	total: 40s	remaining: 0us
bestTest = 4.884587648
bestIteration = 40
Shrink model to first 41 iterations.
Clipped RMSE 0.37297401685751475


In [ ]:
# Let's experiment with faster learning rate and less iterations to roughly find sensible hyperparameters

# parameters to tune: depth, bootstrap_type, bagging temprerature (for Bayesian bootstrap), subsample
# grow_policy -> min_data_in_leaf, max_leaves



In [ ]:
res_dict={}
N_RAND=10
for d in range(1,17):
    print('depth: ',d)
    sum = 0
    for r in range(N_RAND):
        reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=d, eta=0.3,metric_period=20, random_seed = r)
        eval_dataset= Pool(X_val,y_val)
        reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
        pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
    #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))
        sum += clipped_rmse(y_val, pred_val)
        
    res_dict[d] = sum / N_RAND
  
print(res_dict)

# there's about +- 0.010 random fluctuation in scores of individual runs
# maybe simple ensembling already on first level to avoid bad luck?


# {1: 0.41843556427060136, 2: 0.39200723272663457, 3: 0.38961572543553347, 4: 0.39156671290508477, 5: 0.3891002776724449, 6: 0.386308048622023, 7: 0.3857013124069831, 8: 0.38184490124245063, 9: 0.3778682006343534, 10: 0.3755674367385847, 11: 0.37650542801215753, 12: 0.37621294975325814, 13: 0.37367612525383515, 14: 0.3740070625371101, 15: 0.37319345260938475, 16: 0.3703305527864274}
# it seems that maximum available depth of 16 is a reasonable choice



In [ ]:
res_dict={}
N_RAND=10
for d in range(1,6):
    print('depth: ',d)
    sum = 0
    for r in range(N_RAND):
        reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=d, eta=0.3,metric_period=20, random_seed = r)
        eval_dataset= Pool(X_val,y_val)
        reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
        pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
    #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))
        sum += clipped_rmse(y_val, pred_val)
        
    res_dict[d] = sum / N_RAND
  
print(res_dict)

# {6: 0.37218429090880945, 7: 0.37242307494125265, 8: 0.37353756761422685, 9: 0.3725724791167392, 10: 0.37342989782446107, 11: 0.37119154253120795, 12: 0.3741465021829831, 13: 0.37239708871922106, 14: 0.3714940959972749, 15: 0.37278789161405324, 16: 0.37291028574714297}

# there's no indication that a greater depth than 6 would be beneficial
# there's about +- 0.010 random fluctuation in scores of individual runs
# maybe simple ensembling already on first level to avoid bad luck?




In [ ]:
res_dict={}
N_RAND=10
for s in np.linspace(0.5,1.0,20):
    print('subsample: ',s)
    sum = 0
    for r in range(N_RAND):
        reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=16, eta=0.3,metric_period=20, subsample=s, bootstrap_type='Bernoulli', random_seed = r)
        eval_dataset= Pool(X_val,y_val)
        reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
        pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
    #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))
        sum += clipped_rmse(y_val, pred_val)
        
    res_dict[s] = sum / N_RAND
  
print(res_dict)

# {0.5: 0.37720847394133683, 0.5263157894736842: 0.37968235475551115, 0.5526315789473684: 0.373719447235152, 0.5789473684210527: 0.3738004089195496, 0.6052631578947368: 0.3801445592900391, 0.631578947368421: 0.3764996905535196, 0.6578947368421053: 0.3756872315081908, 0.6842105263157895: 0.3803532474804272, 0.7105263157894737: 0.3752644353767258, 0.7368421052631579: 0.37537710048459927, 0.763157894736842: 0.37466870958854703, 
# 0.7894736842105263: 0.37254016810717994, 0.8157894736842105: 0.3770991314953297, 0.8421052631578947: 0.3736975968544275, 0.868421052631579: 0.3782591167219179, 0.8947368421052632: 0.3757776519133261, 0.9210526315789473: 0.3753221768209526, 0.9473684210526315: 0.37274794779142245, 0.9736842105263157: 0.3721892562033758, 1.0: 0.3712936972609858}

# Bernoulli Sampling seems to work ok, but does not bring marked improvements
# the value of subsample parameter seems to have little effect if at all


In [ ]:
res_dict={}
N_RAND=10
for s in np.linspace(0.5,1.0,20):
    print('subsample: ',s)
    sum = 0
    for r in range(N_RAND):
        reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=16, eta=0.3,metric_period=20, subsample=s, bootstrap_type='Poisson', random_seed = r)
        eval_dataset= Pool(X_val,y_val)
        reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
        pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
    #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))
        sum += clipped_rmse(y_val, pred_val)
        
    res_dict[s] = sum / N_RAND
  
print(res_dict)

# roughly similar results as in 'Bernoulli' case
# no need to use this


In [ ]:
res_dict={}
N_RAND=10
for t in np.logspace(-3,3,20):
    print('subsample: ',s)
    sum = 0
    for r in range(N_RAND):
        reg=CatBoostRegressor(task_type='GPU', iterations=120, depth=16, eta=0.3,metric_period=20, bagging_temperature=t, bootstrap_type='Bayesian', random_seed = r)
        eval_dataset= Pool(X_val,y_val)
        reg.fit(X_train.to_numpy(), y_train, eval_set=eval_dataset)
        pred_val = np.clip(reg.predict(X_val.to_numpy()), 0, 20)
    #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))
        sum += clipped_rmse(y_val, pred_val)
        
    res_dict[s] = sum / N_RAND
  
print(res_dict)


Experiment with number of iterations: 

reg=CatBoostRegressor(task_type='GPU', iterations=1200, depth=16, eta=0.03,metric_period=20)
bestTest = 4.89748216
bestIteration = 740
Shrink model to first 741 iterations.
Clipped RMSE 0.3629817122046768



In [8]:
reg=CatBoostRegressor(task_type='GPU', iterations=740, depth=16, eta=0.03,metric_period=20)
reg.fit(X_trainval.to_numpy(), y_trainval)
pred_test = np.clip(reg.predict(X_test.to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-catboost-feature_set_basic.csv')
# w/ all previous months 0.989679 and 0.994916
# w/ 12 past month examples LB 0.996330 and 0.993701

0:	learn: 4.1324599	total: 356ms	remaining: 4m 23s
20:	learn: 3.5702479	total: 6.81s	remaining: 3m 53s
40:	learn: 3.1942986	total: 13.1s	remaining: 3m 44s
60:	learn: 2.9399711	total: 19.5s	remaining: 3m 37s
80:	learn: 2.7633477	total: 26s	remaining: 3m 31s
100:	learn: 2.6503653	total: 32.4s	remaining: 3m 24s
120:	learn: 2.5586229	total: 38.7s	remaining: 3m 18s
140:	learn: 2.4876473	total: 45.2s	remaining: 3m 11s
160:	learn: 2.4200432	total: 51.3s	remaining: 3m 4s
180:	learn: 2.3659357	total: 57.7s	remaining: 2m 58s
200:	learn: 2.3251583	total: 1m 4s	remaining: 2m 51s
220:	learn: 2.2851810	total: 1m 10s	remaining: 2m 45s
240:	learn: 2.2526937	total: 1m 16s	remaining: 2m 38s
260:	learn: 2.2252402	total: 1m 23s	remaining: 2m 32s
280:	learn: 2.1876598	total: 1m 29s	remaining: 2m 26s
300:	learn: 2.1530316	total: 1m 35s	remaining: 2m 19s
320:	learn: 2.1291504	total: 1m 42s	remaining: 2m 13s
340:	learn: 2.0988341	total: 1m 48s	remaining: 2m 6s
360:	learn: 2.0775948	total: 1m 54s	remaining: 2m

X_test

In [10]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':3, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
              }
model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb_val = np.clip(model.predict(X_val), 0, 20)
print('Clipped RMSE of lgb predictions is ', clipped_rmse(y_val, pred_lgb_val))
model = lgb.train(lgb_params, lgb.Dataset(X_trainval, label=y_trainval), 100)
pred_lgb_test = np.clip(model.predict(X_test), 0, 20)
write_predictions_by_array(pred_lgb_test[submissionidx2testidx], 'submission-lgb-feature_set_basic.csv')

Clipped RMSE of lgb predictions is  0.374919455274064


Catboost seems to give comparable performance -> use it

In [ ]:
X_test.columns.values

# 2) Feature set 2: text based features

In [ ]:
# load data

all_data = pd.read_csv(os.path.join(DATA_FOLDER, 'feature_set_text.csv'))

dates=all_data['date_block_num']

y_train = all_data.loc[(dates>= date_block_val - 9) & (dates<= date_block_val - 2), 'target']
y_trainval = all_data.loc[(dates>= date_block_test - 9) & (dates<= date_block_test - 2), 'target']
y_val = all_data.loc[dates == date_block_val, 'target']
y_test = all_data.loc[dates == date_block_test, 'target']

to_drop_cols = ['target','date_block_num']

X_train = all_data.loc[(dates>= date_block_val - 9) & (dates<= date_block_val - 2)].drop(to_drop_cols, axis=1)
X_trainval = all_data.loc[(dates>= date_block_test - 9) & (dates<= date_block_test - 2)].drop(to_drop_cols, axis=1)
X_val = all_data.loc[dates == date_block_val].drop(to_drop_cols, axis=1)
X_test = all_data.loc[dates == date_block_test].drop(to_drop_cols, axis=1)

shop_item2submissionid={}
for idx, row in test_spec.iterrows():
    shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])] = row['ID']
    
test_data=all_data.loc[dates == date_block_test, ['shop_id','item_id']]    
    
testidx2submissionidx=np.zeros(test_data.shape[0], dtype=np.int32)
for idx in range(test_data.shape[0]):
    row =test_data.iloc[idx]
    testidx2submissionidx[idx] = shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])]
    
 
#invert the mapping
submissionidx2testidx=np.zeros(test_data.shape[0], dtype=np.int32)
for i in range(test_data.shape[0]):
    submissionidx2testidx[testidx2submissionidx[i]]=i
    
del test_data
gc.collect()    



In [ ]:
from sklearn import linear_model

#model=linear_model.RidgeCV(alphas=np.logspace(-3,13), fit_intercept=True, normalize=True)
model=linear_model.Ridge(alpha=0.04, fit_intercept=True, normalize=True)
model.fit(X_train.to_numpy(), y_train)
pred_val = np.clip(model.predict(X_val.to_numpy()), 0, 20)
#print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))

model.fit(X_trainval.to_numpy(), y_trainval)
pred_test = np.clip(model.predict(X_test.to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-ridge-feature_set_text.csv')
# LB 1.203418 and 1.189612


In [ ]:
model.alpha_

# Feature set 3: lags within shop

In [ ]:
# load data

all_data = pd.read_csv(os.path.join(DATA_FOLDER, 'feature_set_within.csv'))

dates=all_data['date_block_num']

y_train = all_data.loc[(dates>= date_block_val - 9) & (dates<= date_block_val - 2), 'target']
y_trainval = all_data.loc[(dates>= date_block_test - 9) & (dates<= date_block_test - 2), 'target']
y_val = all_data.loc[dates == date_block_val, 'target']
y_test = all_data.loc[dates == date_block_test, 'target']

to_drop_cols = ['target','date_block_num']

X_train = all_data.loc[(dates>= date_block_val - 9) & (dates<= date_block_val - 2)].drop(to_drop_cols, axis=1)
X_trainval = all_data.loc[(dates>= date_block_test - 9) & (dates<= date_block_test - 2)].drop(to_drop_cols, axis=1)
X_val = all_data.loc[dates == date_block_val].drop(to_drop_cols, axis=1)
X_test = all_data.loc[dates == date_block_test].drop(to_drop_cols, axis=1)

shop_item2submissionid={}
for idx, row in test_spec.iterrows():
    shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])] = row['ID']
    
test_data=all_data.loc[dates == date_block_test, ['shop_id','item_id']]    
    
testidx2submissionidx=np.zeros(test_data.shape[0], dtype=np.int32)
for idx in range(test_data.shape[0]):
    row =test_data.iloc[idx]
    testidx2submissionidx[idx] = shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])]
    
 
#invert the mapping
submissionidx2testidx=np.zeros(test_data.shape[0], dtype=np.int32)
for i in range(test_data.shape[0]):
    submissionidx2testidx[testidx2submissionidx[i]]=i
    
del test_data
gc.collect()    



In [ ]:
from sklearn import linear_model

#model=linear_model.RidgeCV(alphas=np.logspace(-3,13), fit_intercept=False)
model=linear_model.Ridge(alpha=3e7, fit_intercept=False)
model.fit(X_train.to_numpy(), y_train)
pred_val = np.clip(model.predict(X_val.to_numpy()), 0, 20)
#print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
print('Clipped RMSE {}'.format(clipped_rmse(y_val, pred_val)))

model.fit(X_trainval.to_numpy(), y_trainval)
pred_test = np.clip(model.predict(X_test.to_numpy()), 0, 20)
write_predictions_by_array(pred_test[submissionidx2testidx], 'submission-ridge-feature_set_within.csv')
# LB 1.215079 and 1.202396


In [ ]:
model.alpha_